In [3]:
#!/usr/bin/env python

# PuMA log and TOAs extractor

### Python script for TOAs extraction and .log file generation.

This script is intended to gain control and check systematics in all PuMA observations. Can be adapted for particular projects though. It is expected to run after the pulsar_reduc routine. It works under the pulsar_reduc organisation scheme: /pulsar_name/yearmonthday/AX/ where AX could be A1 or A2 with (a) pulsar .fil files (b) calibration .fil files (c) .pfd files (d) .mask files.


Version: 0.1
Date: 1/4/19
authors: Luciano Combi & Paula Kornecki

In [ ]:
# Import standard packages
import numpy as np
import matplotlib.pyplot as mp
import astropy 
from astropy.io import ascii
from astropy.table import Table, Column, MaskedColumn

# We will need to execute shell scripts

import sys 
import glob
import os
sys.path.append('/opt/pulsar_software/presto-stockert/bin/')
import shutil
import getopt


# Import psrchive and rfifind libraries

import psrchive 
import rfifind
from sigproc import *

In [ ]:
# Let us enable a no timing option and a help. Future options of calibration an pazi

import argparse
parser = argparse.ArgumentParser()
parser.add_argument("-n", "--notoa", action="store_true",
                    help="save log and moves folders without obtaining TOAs")
args = parser.parse_args()

if args.notoa:
    antenna = 'N'
    print('Saving log and moving files without obtaining toas')
else: antenna = 'Y'

In [ ]:
# Extract number of parts of raw files (This would be unnecesary if the adquisition soft. worked properly) and antenna information

rawname = os.popen('ls *.fil | grep -v *pul_cal*.fil').read().split()
nobs = len(rawname)
firstraw = os.popen('header '+rawname[0]).read().split('\n')

# We should change 'machine' to 'telescope' once we re-compile PRESTO 
telid = firstraw[5].split(':')[1].strip()
scoord = firstraw[6].split(' ')[19].strip()+'00'+firstraw[7].split(' ')[18].strip()+'00'
tsample = firstraw[15].split(':')[1].strip()

In [ ]:
# Open pfd file inside folder. Only one PSR pfd expected. Pazi not included yet
# We use glob but we might as well use os.popen(ls).

pfdname = glob.glob('*PSR*.pfd')[0]
maskname = glob.glob('*.mask')[0]

In [ ]:
# Correct metadata information. We should include a proper name function here (e.g. for vela)

# Coord
os.popen('psredit'+' -c coord='+scoord+' -m '+pfdname).read()

# Reciever
if telid == 'RTL_Filterbank':
    os.popen('psredit'+' -c rcvr:name=AII'+' -m '+ pfdname).read()
else:
    os.popen('psredit'+' -c rcvr:name=AI'+' -m '+ pfdname).read()

    # Backend
os.popen('psredit'+' -c be:name=EB120'+ ' -m '+ pfdname).read()
# Proyect
os.popen('psredit'+' -c obs:projid=PuMA'+ ' -m '+ pfdname).read()


In [ ]:
# In this step we should include a zapping option and calibration. 
# Since we do not have yet two polarization we skip calibration.

In [ ]:
# Save all relevant data from the pfd using rfifind and psrchive

arch = psrchive.Archive_load(pfdname)
maskrfi = rfifind.rfifind(maskname)
mjd = arch.start_time()

#Usable % of the observations due to RFI. We do not take into account pazi filters.

maskarr = maskrfi.bytemask
nbadint = float(np.count_nonzero(maskarr))
ntotalint = float(len(maskrfi.goodints))*float(len(maskrfi.freqs))

# Parameters (we also have nobs and coords)

datemjd = [mjd.in_days()]
pulsar =  ['J'+arch.get_source()[4:]]
antenna = [arch.get_receiver_name()]
bw = [arch.get_bandwidth()]
freq = [arch.get_centre_frequency()]
nbin = [arch.get_nbin()]
nchan = [arch.get_nchan()]
pol = [arch.get_npol()]
calyn = [arch.get_poln_calibrated()]
snr = [os.popen('psrstat -jTFp -Q -q -c snr '+pfdname).read().strip('\n').strip(' ')]
obstime = [arch.integration_length()/60]
rfitime = [maskrfi.dtint]
usablepercent = [nbadint/ntotalint*100]

logvar = [pulsar, datemjd, antenna, scoord, bw, freq, nbin, nchan, pol, calyn, snr, obstime, rfitime, usablepercent]
lognames = ['Name of pulsar', 'Date', 'Coordinates' 'Antenna', 'BW', 'Freq','NBin','NFchanel','NPol','Calibrated?','S/R','Observation Time','Time of RFI Mask','% of RFI in obs']

In [ ]:
# Check if we are doing timing. Check if folder is created. If not, create.

if timing == 'N':
    destination = './../../pobs_notim/'
else:
    destination = './../../pobs_tim/'
    
if not os.path.exists(destination):
    os.makedirs(destination)

In [ ]:
# Copy pfd and polycos to processobs folder (should move calibrated files in the future)

if timing == 'N':
    os.system('cp ./'+pfdname+' ./../../pobs_notim/')
    os.system('cp ./'+maskname+' ./../../pobs_tim/')
else:
    os.system('cp ./'+maskname+' ./../../pobs_tim/')
    os.system('cp ./'+pfdname+' ./../../pobs_tim/')
    os.system('cp ./'+pfdname+'.polycos'+' ./../../pobs_tim/')

In [ ]:
# Create a new template for timing with existing observations.

if not timing == 'N':
    os.system('psradd -F -P 'pfdname'.std *.pfd')

In [ ]:
# Do timing. If there is a preferred template, use that one and call it puslsarimp.std.

if os.path.exist('../../pobs_tim/'+pulsar+'_imp.std'):
    template = pulsar+'imp.std'
    print('Using imported profile')
else:
    template = pulsar+'.std'

def pat_tim(a):
    if a == 1:
    os.system('pat -F -j "T '+str(a)+'"'+' -f "tempo2" -s ../../pobs_tim/'+' '+template+' >> ../../pobs_tim/total'+pulsar+'.tim')
    elif not a ==1:
        os.system('pat -F -j "T '+str(a)+'"'+' -f "tempo2" -s ../../pobs_tim/'+' '+template' >> ../../pobs_tim/total'+str(a)+pulsar+'.tim')
        os.system('pat -F -j "T '+str(a)+'"'+' -f "tempo2" -s ../../pobs_tim/'+' '+pfdname' > total'+str(a)+pulsar+'.tim')
    
if not timing == 'Y':
    pat_tim(1) and pat_tim(10)

In [ ]:
# Do the log with the current information. Use astropy table. 

file_name = pulsar[0]+'log.txt'
if not os.path.exists(destination+file_name):
    table = Table(logvar, names = lognames)
    ascii.write(table,destination+file_name)
    print ('*** A new pulsar log table for '+ file_name+ ' has been created ***') 
else:
    table = ascii.read(destination+file_name)
    table.add_row(logvar)
    ascii.write(table,destination+file_name)    